In [ ]:
import netCDF4 as nc
import numpy as np

file1 = nc.Dataset('data1.nc')
file2 = nc.Dataset('data2.nc')
print(file1)

lat = file2['latitude'][30]
print(lat) # 위도가 0.25 단위
# 우리는 37.5도 기준으로 한다.
lon = file2['longitude'][28]
print(lon)
# 우리는 127도 기준으로 한다.

print(file1['sst'])

# wind speed(m/s)
# 10m x-dir wind speed(m/s)
u1 = file1['u10'][:, 30, 28]
u2 = file2['u10'][:, 30, 28]
u = list(u1) + list(u2)
# 10m y-dir wind speed(m/s)
v1 = file1['v10'][:, 30, 28]
v2 = file2['v10'][:, 30, 28]
v = list(v1) + list(v2)
# 우리는 풍속이 궁금하므로 피타고라스정리 통해 구한다.
wind_spd = []
for i in range(len(u)):
    wind_spd.append(np.sqrt(u[i]**2 + v[i]**2))

# 2m dew point temperature(K->C)
dt1 = file1['d2m'][:, 30, 28] -273.15
dt2 = file2['d2m'][:, 30, 28] -273.15
dt = list(dt1) + list(dt2) 

# 2m temperature(K -> C)
t1 = file1['t2m'][:, 30, 28] -273.15
t2 = file2['t2m'][:, 30, 28] -273.15
t = list(t1) + list(t2)

# humidity (temp - dewpoint)
h = list(t1 - dt1) + list(t2 - dt2)

# low cloud cover - 단위 없이 0~1사이로 표현
lcc1 = file1['lcc'][:, 30, 28]
lcc2 = file2['lcc'][:, 30, 28]
lcc = list(lcc1) + list(lcc2)
  
# surface air pressure(Pa -> hPa)
sp1 = file1['sp'][:, 30, 28] / 100
sp2 = file2['sp'][:, 30, 28] / 100
sp = list(sp1) + list(sp2)

# sea surface temperature(K -> C)
# 그런데 해수 온도는 한 지점만 활용하는 것이 아니라 위도 35~37.5, 경도 123~125의 자료의 평균 활용할 것이다.
sst_lat = file1['latitude'][30:41] 
sst_lon = file1['longitude'][20:29] 
print(sst_lat)
print(sst_lon)
sst1 = file1['sst'][:, 30:41, 20:29] -273.15
sst2 = file2['sst'][:, 30:41, 20:29] -273.15
sst = list(sst1) + list(sst2)
sst_avg = []
for i in range(len(sst)):
    sst_avg.append(np.mean(sst[i][:][:]))


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2021-10-14 19:45:58 GMT by grib_to_netcdf-2.23.0: /opt/ecmwf/mars-client/bin/grib_to_netcdf -S param -o /cache/data6/adaptor.mars.internal-1634240747.382126-5540-14-04b0e7a6-97fb-4e77-be78-b06ebb28d2f9.nc /cache/tmp/04b0e7a6-97fb-4e77-be78-b06ebb28d2f9-adaptor.mars.internal-1634240395.0307324-5540-3-tmp.grib
    dimensions(sizes): longitude(61), latitude(61), time(589)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 u10(time, latitude, longitude), int16 v10(time, latitude, longitude), int16 d2m(time, latitude, longitude), int16 t2m(time, latitude, longitude), int16 lcc(time, latitude, longitude), int16 sst(time, latitude, longitude), int16 sf(time, latitude, longitude), int16 sp(time, latitude, longitude), int16 tp(time, latitude, longitude)
    groups: 
37.5
127.0
<class 'netCDF4._netCDF4

In [ ]:
# SOI 지수도 가져온다.
import csv

SOI = []
with open('SOI.csv', 'r') as file:
    SOI_data = csv.reader(file)
    for row in SOI_data:
        SOI.append(row[1])
SOI = SOI[1:]  # 맨 첫줄은 headerline
for i in range(len(SOI)):
    SOI[i] = float(SOI[i])

In [ ]:
# snow fall(m)
# 본 데이터는 시간당 몇 m 기준으로 나와있는 것이다.
# 그렇기 때문에 하루동안의 적설량을 따로 구해줘야한다.

snow_file1 = nc.Dataset('snow1.nc')
snow_file2 = nc.Dataset('snow2.nc')

print(snow_file1['sf'])
snow_lat = snow_file1['latitude'][2]  # 우리가 기준이 되는 위도는 37.5도
print(snow_lat)
snow_lon = snow_file1['longitude'][2] # 우리가 기준이 되는 경도는 37.5도
print(snow_lon)

sf1 = snow_file1['sf'][:, 2, 2]
sf2 = snow_file2['sf'][:, 2, 2]
sf = list(sf1) + list(sf2)

# 일별 시간당 적설량 데이터를 더해주는 함수
def daily_snowfall(data):
    daily_snow = []
    for i in range(len(data)//24):
        daily_snow.append(np.sum(data[24*i : 24*i +24]) * 100) # 단위를 m에서 cm로 변환
    return daily_snow
snowfall = daily_snowfall(sf)


# 일별 눈이 왔는지 안왔는지 판단하는 함수
# 0.01cm 이상 와야지 눈이 왔다고 판단
def decision_snow(data):
    decision = []
    for i in range(len(data)):
        if data[i] > 0.001:
            decision.append(1)
        else:
            decision.append(0)
    return(decision)
snow_decision = decision_snow(snowfall)    

<class 'netCDF4._netCDF4.Variable'>
int16 sf(time, latitude, longitude)
    scale_factor: 3.85588834725697e-08
    add_offset: 0.0012634203758622189
    _FillValue: -32767
    missing_value: -32767
    units: m of water equivalent
    long_name: Snowfall
    standard_name: lwe_thickness_of_snowfall_amount
unlimited dimensions: 
current shape = (14136, 5, 5)
filling on
37.5
127.0


In [ ]:
def divide_by_year(data):
    data_by_year = []
    for i in range(len(data)//31):
        data_by_year.append(data[31*i : 31*i + 31])
    return data_by_year

wind_spd = divide_by_year(wind_spd)
h = divide_by_year(h)
t = divide_by_year(t)
lcc = divide_by_year(lcc)
sp = divide_by_year(sp)
sst_avg = divide_by_year(sst_avg)
snow_decision = divide_by_year(snow_decision)

In [ ]:
# 우리가 1960~2020년까지 총 61년에 걸친 데이터를 이용하지만 크리스마스는 1년에 1번밖에 없는 관계로 전체 분석도 61번밖에 못하는 문제가 생긴다.
# 어차피 12월의 강설 경향성을 아는 것이 중요하므로 25일 직전 5일 동안의 데이터(20~24일)과 20일 직전 5일 동안의 데이터(14~19일),30일 직전 5일 동안의 데이터(24~29일)로 나누어서 생각해보자.

def make_mean(data):
    mean20 = []
    mean25 = []
    mean30 = []
    for i in range(61): # 61년 동안
        mean20.append(np.mean(data[i][14:19]))
        mean25.append(np.mean(data[i][19:24]))
        mean30.append(np.mean(data[i][24:29]))
    return mean20, mean25, mean30

wind_spd20, wind_spd25, wind_spd30 = make_mean(wind_spd)
h20, h25, h30 = make_mean(h)
t20, t25, t30 = make_mean(t)
lcc20, lcc25, lcc30 = make_mean(lcc)
sp20, sp25, sp30 = make_mean(sp)
sst_avg20, sst_avg25, sst_avg30 = make_mean(sst_avg)

print(wind_spd20)

[1.9348141940723445, 2.368797121583873, 1.0224772306855687, 2.0092396894637323, 1.9174805907269135, 3.1818807069858237, 1.6045330908661684, 2.145216986804139, 1.7225164446984529, 1.6336057849022119, 2.3740543751993215, 3.1471094231634966, 1.9623694751305503, 2.061290372296144, 1.7521642134974293, 2.0016490660317214, 3.215760308396123, 2.1178791126052667, 2.341243760568962, 2.4886500092436528, 1.475433658112919, 1.4939194212382816, 2.036710408897118, 2.1343675691228614, 1.730033204175665, 2.517644216802327, 2.5978454809819382, 2.2041831796770373, 2.1709904239143647, 1.6937092577464956, 1.5272569804319291, 1.5784805767575345, 1.348547137303744, 2.467250464480061, 2.0372090225073336, 1.6908625839263465, 2.436856346521514, 1.6208461309082527, 2.064374981794524, 3.452184679255102, 2.454206720670328, 2.628745671847361, 1.8966056117622103, 3.4912484276516436, 1.773286782583805, 1.7718432563755502, 1.8886667613499548, 0.9653282982039461, 2.313814540127352, 3.6849644270235573, 2.627087442425933

In [ ]:
final_data = []
for i in range(61):
    final_data.append([wind_spd20[i], h20[i], t20[i], lcc20[i], sp20[i], sst_avg20[i], SOI[i], snow_decision[i][19]])
for i in range(61):
    final_data.append([wind_spd30[i], h30[i], t30[i], lcc30[i], sp30[i], sst_avg30[i], SOI[i], snow_decision[i][29]])
for i in range(61):
    final_data.append([wind_spd25[i], h25[i], t25[i], lcc25[i], sp25[i], sst_avg25[i], SOI[i], snow_decision[i][24]])
    
    
train_data = final_data[:160]
test_data = final_data[160:]

train = 'train.csv'
f = open(train, 'w')
f.write('wind_spd, humidity, temp, low_cloud, pressure, sea_temp, SOI, snow\n')
for i in range(len(train_data)):
    f.write(str(train_data[i][:])[1:-1] + '\n')  # 양 옆에 list를 표현하는 []가 csv파일에 같이 끌려들어가는 것을 막기 위해 [1:-1]만 가져간다.
f.close()

test = 'test.csv'
f = open(test, 'w')
f.write('wind_spd, humidity, temp, low_cloud, pressure, sea_temp, SOI, snow\n')
for i in range(len(test_data)):
    f.write(str(test_data[i][:])[1:-1] + '\n')  # 양 옆에 list를 표현하는 []가 csv파일에 같이 끌려들어가는 것을 막기 위해 [1:-1]만 가져간다.
f.close()